In [ ]:
import ee
import datetime
import pandas as pd
import csv

ee.Authenticate()
ee.Initialize()

def get_elevation_slope(lat, lon):
    point = ee.Geometry.Point([lon, lat])

    elevation_data = ee.Image('USGS/SRTMGL1_003')

    slope_data = ee.Terrain.slope(elevation_data)

    # Get the elevation value for the point
    elevation_samples = elevation_data.sample(point, 30)  # 30 m native resolution of the dataset, to get the most precise value
    if elevation_samples.size().getInfo() == 0:
        elevation = None
    else:
        elevation = elevation_samples.first().get('elevation').getInfo()

    # Get the slope value for the point
    slope_samples = slope_data.sample(point, 30)
    if slope_samples.size().getInfo() == 0:
        slope = None
    else:
        slope = slope_samples.first().get('slope').getInfo()

    return elevation, slope

# Function to save the results to a CSV file
def save_to_csv(filename, data):
    with open(filename, mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Latitude', 'Longitude', 'Elevation', 'Slope'])

        for row in data:
            writer.writerow(row)

# Read the CSV file
df = pd.read_csv("mapdata_wildfires.csv")

coordinates = []

for _, row in df.iterrows():
    lat = row['incident_latitude']
    lon = row['incident_longitude']

    # Create a tuple and append it to the list
    coordinates.append((lat, lon))

results = []

# Iterate through the coordinate pairs and get the elevation and slope values
for lat, lon in coordinates:
    elevation, slope = get_elevation_slope(lat, lon)
    print(f'Latitude: {lat}, Longitude: {lon}, Elevation: {elevation}, Slope: {slope}')
    results.append((lat, lon, elevation, slope))

# Save the results to a CSV file
save_to_csv('elevation_slope_data_wildfires.csv', results)


In [2]:
print("The results were saved to a CSV file.")

The results were saved to a CSV file.
